In [1]:
import os

# Désactiver le GPU en définissant CUDA_VISIBLE_DEVICES à un vide#
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer

import RktnChallenge.preprocessing.tokenizeString
importlib.reload(RktnChallenge.preprocessing.tokenizeString)
from RktnChallenge.preprocessing.tokenizeString import tokenizeString

import RktnChallenge.preprocessing.filterStopWords
importlib.reload(RktnChallenge.preprocessing.filterStopWords)
from RktnChallenge.preprocessing.filterStopWords import filterStopWords


import RktnChallenge.preprocessing.mergeFeatures
importlib.reload(RktnChallenge.preprocessing.mergeFeatures)
from RktnChallenge.preprocessing.mergeFeatures import mergeFeatures

import RktnChallenge.preprocessing.mostOccur
importlib.reload(RktnChallenge.preprocessing.mostOccur)
from RktnChallenge.preprocessing.mostOccur import mostOccur

import RktnChallenge.preprocessing.Dropper
importlib.reload(RktnChallenge.preprocessing.Dropper)
from RktnChallenge.preprocessing.Dropper import Dropper

import RktnChallenge.preprocessing.TokenListToString
importlib.reload(RktnChallenge.preprocessing.TokenListToString)
from RktnChallenge.preprocessing.TokenListToString import TokenListToString

2023-09-06 00:00:41.419306: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-06 00:00:41.798533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 00:00:43.174387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
RktnModel = ModelTrainer("data_tr.csv")

Removing StopWords

In [ ]:
On crop puis on merge

On merge et on crop ensuite

In [3]:
addendum = ['a','à','<p>','<b>','<div>','<em>','<br>']

#on tokenise et retire les stopwords designation

desi_tokenize= tokenizeString("designation_tkn","designation")
desi_stopwords = filterStopWords("designation_tkn", "designation_tkn"
                                 ,lang=["english","french"],addendum=addendum)


#on tokenise et retire les stopwords description
desc_tokenize= tokenizeString("description_tkn","description")
desc_stopwords = filterStopWords("description_tkn", "description_tkn"
                                 ,lang=["english","french"],addendum=addendum)


merge_desi_desc = mergeFeatures("merged","designation_tkn","description_tkn")
desc_mostOccur= mostOccur("merged_desi_desc","merged",100,100)


toDrop = ["designation_tkn","description_tkn","merged"]
cleanDropper = Dropper(column_to_drop = toDrop)
toString = TokenListToString("merged_desi_desc","merged_desi_desc")

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("tokenize designation",desi_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter designation",desi_stopwords)  # filtrage des stopwords
RktnModel.addPreprocessStep("tokenize description",desc_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter description",desc_stopwords)  # filtrage des stopwords
RktnModel.addPreprocessStep("merge designation and desc",merge_desi_desc) # merge desi&desc
RktnModel.addPreprocessStep("desc reduce",desc_mostOccur)         # Limite en taille
RktnModel.addPreprocessStep("tostring",toString) # merge desi&desc

#RktnModel.addPreprocessStep("cleaning",cleanDropper) # on clean les colonnes crées
RktnModel.preprocess()



[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Vectorization des données

In [4]:
_X = RktnModel.data["merged_desi_desc"]

max_features = 13000

RktnModel.create_vectorizer(name="tfidf", X = _X, ngram_range=(1,3),max_features=max_features)
X_data = RktnModel.vectorizer_transform(_X)     


y = RktnModel.encodeLabel("prdtypecode")
label_size = RktnModel.getLabelSize()

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size = 0.20, random_state = 42)


In [6]:
del RktnModel

Definition d'un modele

In [7]:
import gc

gc.collect()

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRUCell,RNN,GRU, Dense,Dropout,GlobalAveragePooling2D, LSTM, Flatten, Input
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.models import Model





model = Sequential([
    Dense(units=max_features, activation="relu", input_shape=(max_features,)),
    Dropout(0.3),
    Dense(units=6000, activation="relu"),
    Dropout(0.3),
    Dense(units=1024, activation="relu"),
    Dropout(0.3),
    Dense(units=512, activation="relu"),
    Dropout(0.3),
    Dense(units=256, activation="relu"),
    Dropout(0.3),
    Dense(units=label_size, activation="softmax")
])


 
optimizer = AdamW(weight_decay=0.01, learning_rate=0.001) 
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13000)             169013000 
                                                                 
 dropout (Dropout)           (None, 13000)             0         
                                                                 
 dense_1 (Dense)             (None, 6000)              78006000  
                                                                 
 dropout_1 (Dropout)         (None, 6000)              0         
                                                                 
 dense_2 (Dense)             (None, 1024)              6145024   
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               5

2023-09-06 00:07:25.344672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 00:07:25.636586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 00:07:25.637052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 00:07:25.639694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-06 00:07:25.639997: I tensorflow/compile

In [8]:
history = model.fit(X_train, y_train, epochs=10, batch_size=100, validation_split=0.2)

Epoch 1/10


2023-09-06 00:07:39.915842: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f502cb7b450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-06 00:07:39.915901: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2023-09-06 00:07:39.981044: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-06 00:07:40.321404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-06 00:07:40.538750: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


544/544 [==============================] - 44s 75ms/step - loss: 1.2352 - accuracy: 0.6515 - val_loss: 0.8338 - val_accuracy: 0.7713
Epoch 2/10
544/544 [==============================] - 39s 72ms/step - loss: 0.6059 - accuracy: 0.8323 - val_loss: 0.7657 - val_accuracy: 0.7914
Epoch 3/10
544/544 [==============================] - 40s 73ms/step - loss: 0.3884 - accuracy: 0.8891 - val_loss: 0.8081 - val_accuracy: 0.7951
Epoch 4/10
544/544 [==============================] - 40s 73ms/step - loss: 0.2759 - accuracy: 0.9233 - val_loss: 0.9476 - val_accuracy: 0.7952
Epoch 5/10
544/544 [==============================] - 39s 72ms/step - loss: 0.2082 - accuracy: 0.9414 - val_loss: 1.0776 - val_accuracy: 0.7912
Epoch 6/10
544/544 [==============================] - 40s 74ms/step - loss: 0.1735 - accuracy: 0.9518 - val_loss: 1.1961 - val_accuracy: 0.7877
Epoch 7/10
544/544 [==============================] - 40s 74ms/step - loss: 0.1489 - accuracy: 0.9585 - val_loss: 1.3679 - val_accuracy: 0.7902
Epo

max_feature à 1000
si taille = 1 52%
si taille = 2 64%
si taille = 400 76%
si taille = 1000 76%

je fais varier max_feature à 13 000
si taille = 100 79.52
si taille = 1000 78%

Max BoW = 76%